In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1.head(2)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem','TotalCPUtime']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = slurm['JobLen'] * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = waittime['year'] + '-' + waittime['month']

In [ ]:
waittime.head()

In [ ]:
waittime = waittime.loc[waittime.JobLen >= 0]

In [ ]:
waittime.info()

# Job Run-Time Analysis

## Job runtime according to job size

In [ ]:
Runtime_jobsize = waittime.groupby('NNodes', as_index=False).agg({'JobLen':'mean','Jcount':'sum'})
Runtime_jobsize['NNodes'] = Runtime_jobsize['NNodes'].apply(int)
Runtime_jobsize = Runtime_jobsize.sort_values('NNodes')

In [ ]:
Runtime_jobsize1 = Runtime_jobsize.loc[Runtime_jobsize.NNodes <= 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 38, 10
Runtime_jobsize1['NNodes'] = Runtime_jobsize1['NNodes'].apply(str)
Runtime_jobsize1['JobLen'] = Runtime_jobsize1['JobLen'] / 3600
plots = sns.barplot(x ='NNodes',y = 'JobLen',data =  Runtime_jobsize1)
plots.set_title('Mean Run time according to job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('Run time in hours',size = 22)
if not os.path.exists('Runtime_resources'):
    os.makedirs('Runtime_resources')
plt.savefig('Runtime_resources/Runtime_resources_js1.png')

# report purpose

In [ ]:
Runtime_jobsize2 = Runtime_jobsize.loc[Runtime_jobsize.NNodes > 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 38, 10
Runtime_jobsize2['NNodes'] = Runtime_jobsize2['NNodes'].apply(str)
Runtime_jobsize2['JobLen'] = Runtime_jobsize2['JobLen'] / 3600
plots = sns.barplot(x ='NNodes',y = 'JobLen',data =  Runtime_jobsize2)
plots.set_title('Mean Run time according to job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('Run time in hours',size = 22)
if not os.path.exists('Runtime_resources'):
    os.makedirs('Runtime_resources')
plt.savefig('Runtime_resources/Runtime_resources_js2.png')

## Job runtime according to QOS

In [ ]:
runtime_QOS = waittime.groupby('QOS', as_index=False).agg({'JobLen':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
runtime_QOS['QOS'] = runtime_QOS['QOS'].apply(str)
runtime_QOS['JobLen'] = runtime_QOS['JobLen'] / 3600
plots = sns.barplot(x ='QOS',y = 'JobLen',data =  runtime_QOS)
plots.set_title('Mean Run time according to QOS',size = 16)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('QoS',size = 22)
plt.ylabel('Run time in hours',size = 22)
if not os.path.exists('Runtime_resources'):
    os.makedirs('Runtime_resources')
plt.savefig('Runtime_resources/RunTime_QOS 1.png')

## Job runtime according to partition

In [ ]:
runtime_Partition = waittime.groupby('Partition', as_index=False).agg({'JobLen':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
runtime_Partition['Partition'] = runtime_Partition['Partition'].apply(str)
runtime_Partition['JobLen'] = runtime_Partition['JobLen'] / 3600
plots = sns.barplot(x ='Partition',y = 'JobLen',data =  runtime_Partition)
plots.set_title('Mean run time according to different Partitions',size = 16)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Partition',size = 22)
plt.ylabel('Run time in hours',size = 22)
if not os.path.exists('Runtime_resources'):
    os.makedirs('Runtime_resources')
plt.savefig('Runtime_resources/Runtime_Partition1.png')

## Mean Job runtime according to project

In [ ]:
Runtime_project = waittime.groupby('Account', as_index=False).agg({'JobLen':'mean','Jcount':'sum'})
Runtime_project['Account'] = Runtime_project['Account'].apply(int)
Runtime_project = Runtime_project.sort_values('Account')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 80, 10
Runtime_project['Account'] = Runtime_project['Account'].apply(str)
plots = sns.barplot(x ='Account',y = 'JobLen',data =  Runtime_project)
plots.set_title('Mean runtime according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('Runtime_resources'):
    os.makedirs('Runtime_resources')
plt.savefig('Runtime_resources/Runtime_project 1.png')

## Total runtime every year

In [ ]:
def functionruntime(years):
    coun = 0
    df_list = []
    df_list2 = []
    varlist = []
    for i in years:
        temp = waittime[waittime.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False)['JobLen'].sum()
        temp1['JobLen'] = temp1['JobLen'] / 3600
        temp1['JobLen'] = temp1['JobLen'].astype(int)
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        temp2 = j.JobLen.std()
        varlist.append(temp2)
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig, (ax, ax_table) = plt.subplots(nrows=2)
        ax_table.axis('off')
        ax.grid(False)
        ax.set_title("Total Run time every month for the year {} in hours".format(str(l.year[0])),size=18)
        width = .5 
        ax.bar(l.month, l.JobLen, width, color='g')
        fig.set_figheight(10)
        fig.set_figwidth(16)
        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=16,rotation = 0)
        ax.set_xlabel('Months', fontsize=16)
        ax.set_ylabel('Job length in hours', fontsize=16)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        if len(varlist) != 0:
            for index,li in enumerate(varlist):
                if count == 1:
                    break
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
                textstr = '\n'.join((
                r'$SD= %.2f$' % (li, ),))
                ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
                verticalalignment='top', bbox=props)
                plt.subplots_adjust(left=0, bottom=0)
                count = count + 1
                varlist.pop(0)
            plt.show()
            coun = coun + 1
            if not os.path.exists('Runtime_analysis'):
                os.makedirs('Runtime_analysis')
            fig.savefig('Runtime_analysis/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)
                
            

In [ ]:
years = ['2019', '2020', '2021', '2022']
functionruntime(years)

## Job count according to runtime every year

In [ ]:
runtime = waittime.copy()

In [ ]:
def ff(row):
    if row['JobLen'] <= 300:
        val = 1
    else:
        val = 0
    return val
runtime['upto5mins'] = runtime.apply(ff,axis =1)
def ff1(row):
    if row['JobLen'] <= 3600 and row['JobLen'] > 300:
        val = 1
    else:
        val = 0
    return val
runtime['uptoonehour'] = runtime.apply(ff1,axis =1)
def ff2(row):
    if row['JobLen'] <= 21600 and row['JobLen'] > 3600:
        val = 1
    else:
        val = 0
    return val
runtime['uptosix'] = runtime.apply(ff2,axis =1)

def ff3(row):
    if row['JobLen'] > 21600 and row['JobLen'] <= 43200:
        val = 1
    else:
        val = 0
    return val
runtime['uptotwelve'] = runtime.apply(ff3,axis =1)
def ff4(row):
    if row['JobLen'] > 604800:
        val = 1
    else:
        val = 0
    return val
runtime['morethanweek'] = runtime.apply(ff4,axis =1)
def ff5(row):
    if row['JobLen'] == 0:
        val = 1
    else:
        val = 0
    return val
runtime['nowaittime'] = runtime.apply(ff5,axis =1)
def ff6(row):
    if row['JobLen'] > 86400 and row['JobLen']<= 604800:
        val = 1
    else:
        val = 0
    return val
runtime['uptoweek'] = runtime.apply(ff6,axis =1)
def ff7(row):
    if row['JobLen'] > 43200 and row['JobLen']<= 86400:
        val = 1
    else:
        val = 0
    return val
runtime['uptoday'] = runtime.apply(ff7,axis =1)

In [ ]:
def functionruntime2(years):
    coun = 10
    df_list = []
    df_list2 = []
    for i in years:
        temp = runtime[runtime.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False).agg({'Jcount':'sum','upto5mins':'sum','uptoonehour':'sum',
                                                      'uptotwelve':'sum','uptoweek':'sum','uptoday':'sum','morethanweek':'sum','uptosix':'sum' })
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to runtime duration for the year {} ".format(str(l.year[0])),size=22)
        width = .5 
        # ax.bar(l.month, l.Jcount, width, color='w', alpha = 0.8)
        ax.bar(l.month, l.upto5mins, width, color='blue',alpha = 0.5)
        ax.bar(l.month, l.uptoonehour, width,bottom = l.upto5mins, color='green',alpha = 0.5)
        ax.bar(l.month, l.uptosix, width,bottom = l.upto5mins + l.uptoonehour,  color='m',alpha = 0.5)
        ax.bar(l.month, l.uptotwelve, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix,  color='brown',alpha = 0.5)
        ax.bar(l.month, l.uptoday, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve,  color='r',alpha = 0.5)
        ax.bar(l.month, l.uptoweek, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve + l.uptoday , color='yellow',alpha = 0.5)
        ax.bar(l.month, l.morethanweek, width, bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve + l.uptoday + l.uptoweek ,color='k',alpha = 0.5)
        
        fig.set_figheight(18)
        fig.set_figwidth(16)
        # for p in ax.patches:
        #     ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=12)
        ax.set_xlabel('Months', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.subplots_adjust(left=0, bottom=0)
        plt.legend(["Runtime upto 5 minutes",'Runtime from 5 to 60 minutes','Runtime from hour to 6 hours','Runtime from 6 to 12 hours','Runtime from 12 to 24 hours','Runtime from 1 day to 1 week','Runtime for more than a week' ],prop={'size': 22})
        plt.show()
        coun = coun + 1
        if not os.path.exists('Runtime_analysis'):
            os.makedirs('Runtime_analysis')
        fig.savefig('Runtime_analysis/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
years = ['2019', '2020', '2021', '2022']
functionruntime2(years)

## Runtime Histogram for individual project

In [ ]:
runtime.head()

In [ ]:
maxx = runtime.JobLen.values.max()
mean = runtime.JobLen.values.mean()
print(maxx)
print(mean)

In [ ]:
smallerruntime = []
smallerruntime = np.linspace(0, mean, 10,endpoint=False)
smallerruntime = [l.tolist() for l in smallerruntime]
smallerruntime1 = np.linspace(0, smallerruntime[1], 15 ,endpoint=False)
smallerruntime1 = [l.tolist() for l in smallerruntime1]
smallerruntime1 = [ '%.1f' % elem for elem in smallerruntime1 ]
smallerruntime1

In [ ]:
smallerruntime1

In [ ]:
del smallerruntime1[0]

In [ ]:
runtimep2 = runtime[['Account','JobLen']]

In [ ]:
runtimep2.info()

In [ ]:
smallerruntime1 = np.asarray(smallerruntime1, dtype = float)
coun = 0
for idx,row in runtimep2.iterrows():
    for index, elem in enumerate(smallerruntime1):
        if runtimep2.loc[idx,'JobLen'] <= smallerruntime1[0]:
            runtimep2.loc[idx,'Rs 0-{}'.format(smallerruntime1[0])] = 1    
        else:
            runtimep2.loc[idx,'Rs 0-{}'.format(smallerruntime1[0])] = 0   
        prevele = smallerruntime1[index - 1]      
        if runtimep2.loc[idx,'JobLen'] > prevele and runtimep2.loc[idx,'JobLen'] <= elem:
            runtimep2.loc[idx,'Rs {}-{}'.format(prevele,elem)] = 1
        else:
            runtimep2.loc[idx,'Rs {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
runtimep2.head()

In [ ]:
runtimep2 = runtimep2.drop(['Rs 715.1-51.1'], axis=1)
colrts = runtimep2.columns
colrts = colrts.tolist()
colrts

In [ ]:
colrts = runtimep2.groupby('Account', as_index=False)[
 'Rs 0-51.1',
 'Rs 51.1-102.2',
 'Rs 102.2-153.2',
 'Rs 153.2-204.3',
 'Rs 204.3-255.4',
 'Rs 255.4-306.5',
 'Rs 306.5-357.6',
 'Rs 357.6-408.6',
 'Rs 408.6-459.7',
 'Rs 459.7-510.8',
 'Rs 510.8-561.9',
 'Rs 561.9-613.0',
 'Rs 613.0-664.0',
 'Rs 664.0-715.1'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in colrts.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_xlabel('Run time criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_title("number of jobs according to Job length for the Account {}".format(str(count)),size=22)
        width = .5 
        ax = temp.plot(kind = 'bar')
        ax.set_xticklabels(['0-51','51-102','102-152','152-204','204-255','255-306','306-357','357-408','408-459','459-510','510-561','561-613','613-664','664-715'],size = 18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_runtime_sub'):
            os.makedirs('Histogram_runtime_sub')
        fig.savefig('Histogram_runtime_sub/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
parts = []
parts1 = []
parts2 = []
parts3 = []
parts4 = []
parts = np.linspace(0, mean, 10,endpoint=False)
parts1 = np.linspace(mean, maxx, 4)
parts = [l.tolist() for l in parts]
parts1 = [l.tolist() for l in parts1]
parts2.append(np.linspace(parts1[0], parts1[1], 20,endpoint=False))
parts3.append(np.linspace(parts1[1], parts1[2], 10,endpoint=False))
parts4.append(np.linspace(parts1[2], parts1[3], 10))
parts2 = [l2.tolist() for l2 in parts2]
parts2 = list(np.concatenate(parts2).flat)
parts3 = [l3.tolist() for l3 in parts3]
parts3 = list(np.concatenate(parts3).flat)
parts4 = [l4.tolist() for l4 in parts4]
parts4 = list(np.concatenate(parts4).flat)

In [ ]:
parts = parts + parts2 + parts3 + parts4
del parts[0]
parts

In [ ]:
parts = [ '%.2f' % elem for elem in parts ]

In [ ]:
parts = np.asarray(parts, dtype = float)

In [ ]:
runtimep = runtime[['Account','JobLen']]
coun = 0
for idx,row in runtimep.iterrows():
    for index, elem in enumerate(parts):
        if runtimep.loc[idx,'JobLen'] <= parts[0]:
            runtimep.loc[idx,'R 0-{}'.format(parts[0])] = 1    
        else:
            runtimep.loc[idx,'R 0-{}'.format(parts[0])] = 0   
        prevele = parts[index - 1]      
        if runtimep.loc[idx,'JobLen'] > prevele and runtimep.loc[idx,'JobLen'] <= elem:
            runtimep.loc[idx,'R {}-{}'.format(prevele,elem)] = 1
        else:
            runtimep.loc[idx,'R {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
runtimep.head()

In [ ]:
runtimep1 = runtimep.drop(['R 1905703.0-766.19'], axis=1)
col = runtimep1.columns
col = col.tolist()
col

In [ ]:
cc = runtimep1.groupby('Account', as_index=False)['R 0-766.19',
 'R 766.19-1532.38',
 'R 1532.38-2298.58',
 'R 2298.58-3064.77',
 'R 3064.77-3830.96',
 'R 3830.96-4597.15',
 'R 4597.15-5363.34',
 'R 5363.34-6129.53',
 'R 6129.53-6895.73',
 'R 6895.73-7661.92',
 'R 7661.92-39295.94',
 'R 39295.94-70929.95',
 'R 70929.95-102563.97',
 'R 102563.97-134197.99',
 'R 134197.99-165832.01',
 'R 165832.01-197466.03',
 'R 197466.03-229100.04',
 'R 229100.04-260734.06',
 'R 260734.06-292368.08',
 'R 292368.08-324002.1',
 'R 324002.1-355636.12',
 'R 355636.12-387270.13',
 'R 387270.13-418904.15',
 'R 418904.15-450538.17',
 'R 450538.17-482172.19',
 'R 482172.19-513806.21',
 'R 513806.21-545440.22',
 'R 545440.22-577074.24',
 'R 577074.24-608708.26',
 'R 608708.26-640342.28',
 'R 640342.28-703610.31',
 'R 703610.31-766878.35',
 'R 766878.35-830146.39',
 'R 830146.39-893414.42',
 'R 893414.42-956682.46',
 'R 956682.46-1019950.49',
 'R 1019950.49-1083218.53',
 'R 1083218.53-1146486.57',
 'R 1146486.57-1209754.6',
 'R 1209754.6-1273022.64',
 'R 1273022.64-1343320.46',
 'R 1343320.46-1413618.27',
 'R 1413618.27-1483916.09',
 'R 1483916.09-1554213.91',
 'R 1554213.91-1624511.73',
 'R 1624511.73-1694809.55',
 'R 1694809.55-1765107.36',
 'R 1765107.36-1835405.18',
 'R 1835405.18-1905703.0'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in cc.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("number of jobs according to runtimes for the Account {}".format(str(count)),size=22)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('Run time criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels( ['0-766',
 '766-1532',
 '1532-2298',
 '2298-3064',
 '3064-3830',
 '3830-4597',
 '4597-5363',
 '5363-6129',
 '6129-6895',
 '6895-7661',
 '7661-39295',
 '39295-70929',
 '70929-102563',
 '102563-134197',
 '134197-165832',
 '165832-197466',
 '197466-229100',
 '229100-260734',
 '260734-292368',
 '292368-324002',
 '324002-355636',
 '355636-387270',
 '387270-418904',
 '418904-450538',
 '450538-482172',
 '482172-513806',
 '513806-545440',
 '545440-577074',
 '577074-608708',
 '608708-640342',
 '640342-703610',
 '703610-766878',
 '766878-830146',
 '830146-893414',
 '893414-956682',
 '956682-1019950',
 '1019950-1083218',
 '1083218-1146486',
 '1146486-1209754',
 '1209754-1273022',
 '1273022-1343320',
 '1343320-1413618',
 '1413618-1483916',
 '1483916-1554213',
 '1554213-1624511',
 '1624511-1694809',
 '1694809-1765107',
 '1765107-1835405',
 '1835405-1905703'],size = 16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_runtime'):
            os.makedirs('Histogram_runtime')
        fig.savefig('Histogram_runtime/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)